In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
data = pd.read_csv('/content/IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [5]:
english_stops = set(stopwords.words('english'))

In [6]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


<ipython-input-6-64c774daf602>:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


In [7]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
39500    [walerian, borowczyks, la, bete, obviously, re...
38375    [this, may, worst, movie, ever, produced, wors...
10523    [okay, this, favourite, since, i, granted, i, ...
48629    [except, jon, heder, this, guy, tanked, entire...
27476    [after, wit, liveliness, highway, roadkill, i,...
                               ...                        
24520    [if, want, see, true, vile, nature, communism,...
6592     [i, watched, movie, clock, morning, time, day,...
25018    [i, really, understand, movie, aimed, from, ab...
47527    [witchery, might, incoherent, lamentably, scri...
33809    [an, unbelievable, plot, laughable, characters...
Name: review, Length: 40000, dtype: object 

5905     [somebody, owes, ang, lee, apology, actually, ...
22096    [this, movie, scary, scenes, devil, uses, gabr...
37899    [came, across, film, net, fluke, horrified, co...
29263    [being, fan, series, i, thought, bad, movie, w...
31578    [uggh, hanna, barbera, what, lousy, unwatchabl...
 

In [8]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [9]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[35729 56911   926 ...  9488   580  3426]
 [    8   110   152 ...     0     0     0]
 [  719     8  1595 ...     0     0     0]
 ...
 [    1    14   289 ...     0     0     0]
 [13802   135  3446 ...  6061   615   665]
 [  688  1254    41 ...     0     0     0]] 

Encoded X Test
 [[ 1690  6373 12753 ...  2089 45037  8519]
 [    8     3   552 ...     0     0     0]
 [  287   516     4 ...     0     0     0]
 ...
 [22098   262  5491 ...     0     0     0]
 [16423  7176   118 ...     0     0     0]
 [  304   201  8391 ...     0     0     0]] 

Maximum review length:  130


In [10]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [17]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    'models/LSTM.keras',  # Change the file extension to `.keras`
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [18]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.5527 - loss: 0.6748
Epoch 1: accuracy improved from -inf to 0.61102, saving model to models/LSTM.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 65s 198ms/step - accuracy: 0.5529 - loss: 0.6747
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - accuracy: 0.7467 - loss: 0.5517
Epoch 2: accuracy improved from 0.61102 to 0.69565, saving model to models/LSTM.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 65s 206ms/step - accuracy: 0.7466 - loss: 0.5518
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.6426 - loss: 0.6286
Epoch 3: accuracy did not improve from 0.69565
313/313 ━━━━━━━━━━━━━━━━━━━━ 79s 197ms/step - accuracy: 0.6426 - loss: 0.6285
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.7097 - loss: 0.5927
Epoch 4: accuracy improved from 0.69565 to 0.71145, saving model to models/LSTM.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 197ms/step - accuracy: 0.7097 - loss: 0.5927
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━

In [20]:
import numpy as np

# Predict class probabilities
y_pred_probs = model.predict(x_test, batch_size=128)

# Convert probabilities to class labels by taking the index of the highest probability
y_pred = np.argmax(y_pred_probs, axis=1)

# Initialize counter for correct predictions
true = 0

# Compare true and predicted values (ensure y_test is in the correct format)
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

# Print results
print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {:.2f}%'.format(true / len(y_pred) * 100))


79/79 ━━━━━━━━━━━━━━━━━━━━ 6s 71ms/step
Correct Prediction: 5008
Wrong Prediction: 4992
Accuracy: 50.08%


In [22]:
loaded_model = load_model('models/LSTM.keras')

In [23]:
review = str(input('Movie Review: '))


Movie Review: A beautifully directed, poignant and bitter sweet melodrama that tugs at your heart strings. A must watch. Do not be surprised to shed a tear or two as you become deeply engaged with the characters in this movie. 


In [24]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  A beautifully directed poignant and bitter sweet melodrama that tugs at your heart strings A must watch Do not be surprised to shed a tear or two as you become deeply engaged with the characters in this movie 
Filtered:  ['a beautifully directed poignant bitter sweet melodrama tugs heart strings a must watch do surprised shed tear two become deeply engaged characters movie ']


In [25]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)


[[   39  1175   432  2975  3025   995  2484 22746   393  5501    39   113
     34   805   656  5111  3182    36   335  1707  3454    29     3     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0]]


In [26]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
[[0.6338722]]


In [29]:
if result >= 0.6:
    print('positive')
else:
    print('negative')

positive
